# <center> 2차 크롤링 - <점핏> </center>

##### 링크들 들어가서 각각 정보 수집, 자율주행

* 기업이름
* 고용형태 : 신입, 경력
* 연봉 (대체로 면접후 결정 일겁니다, 크롤링 사이트 내용 그대로 검출하면 될듯!)
* 기업주소 : 기업 메인 위치
* 키워드 : 자율주행 / 드론 / 딥러닝 / 로봇 제어</br></br>
 -----뭉쳐서 '주요사항' column-----
* 자격 요건 : ROS, Python, C++, 학력
* 담당업무
* 우대사항

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service("../drive/chromedriver-linux64/chromedriver"), options=options)

driver.get("https://www.jumpit.co.kr/search?locationTag=1&sort=relation&keyword=%EB%94%A5%EB%9F%AC%EB%8B%9D")
time.sleep(2)

# 페이지 끝까지 스크롤
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

html_source = driver.page_source
soup = BeautifulSoup(html_source, "html.parser")
time.sleep(3)

job_box_1 = soup.find('section', class_='sc-c12e57e5-3 gjgpzi')
job_box_2 = job_box_1.find('section', class_='sc-80cff0b9-0 cOzTle')
job_box_3 = job_box_2.find_all('div', class_='sc-c8169e0e-0 gOSXGP')

image_link_list = []
category_list = []
major_work_list = []

for info in job_box_3:

    job_link = info.find('a')['href']
    image_link_list.append(job_link)

    major_work = info.find('h2', class_='position_card_info_title').text
    major_work_list.append(major_work)


    category_list.append('딥러닝')

print(image_link_list, category_list)


In [ ]:
print(len(image_link_list))
print(len(category_list))

In [3]:
import pandas as pd

df = pd.DataFrame({
    '키워드': category_list,
    'url': image_link_list,
    '담당업무' : major_work_list,
})

In [4]:
df.tail()

,키워드,url
143,딥러닝,/position/22207
144,딥러닝,/position/22041
145,딥러닝,/position/1706
146,딥러닝,/position/20640
147,딥러닝,/position/6835


In [5]:
df.to_csv("../data/jumpit_deeplearning_base.csv", sep =",", encoding = 'utf-8')

In [6]:
import pandas as pd

df = pd.read_csv("../data/jumpit_deeplearning_base.csv", encoding='utf-8', index_col=0)

df['url'] = df['url'].apply(lambda x: f"https://www.jumpit.co.kr{x}")


In [1]:
import pandas as pd
df= pd.read_csv("../data/jumpit_deeplearning_base.csv", encoding='utf-8', index_col=0)

In [7]:
df.tail()

,키워드,url
143,딥러닝,https://www.jumpit.co.kr/position/22207
144,딥러닝,https://www.jumpit.co.kr/position/22041
145,딥러닝,https://www.jumpit.co.kr/position/1706
146,딥러닝,https://www.jumpit.co.kr/position/20640
147,딥러닝,https://www.jumpit.co.kr/position/6835


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import requests
import os
from tqdm.notebook import tqdm


options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={generate_user_agent()}") 

driver = webdriver.Chrome(service=Service("../drive/chromedriver-linux64/chromedriver"),options=options)

company_address_list = []
company_name_list=[]
hire_list = []
tech_list=[]
salary_list =[]


for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    driver.get(row['url'] )
    major_work_list_tmp = []

    time.sleep(1)  

    for _ in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, "html.parser")

    name_box = soup.find('section', class_='sc-3a5c981a-0 iORPxB')
    company_name = name_box.find('a').text
    company_name_list.append(company_name)

    company_address_box = soup.find('section', class_='sc-2da322c6-0 hWFbtF')
    company_address_box2 = company_address_box.find_all('dl', class_='sc-2da322c6-1 ksYhFT')

    company_address_list.append(company_address_box2[3].text)
    hire_list.append(company_address_box2[0].text)

    essential_tech_box = soup.find_all('dl', class_='sc-ae955bd-0 MHqtB')

    salary_list.append('회사내규에 따름')

    tech_stack = essential_tech_box[0].find('pre').text
    major_work_list_tmp.append(tech_stack)

    essential_tech1 = essential_tech_box[1].find('pre').text
    major_work_list_tmp.append(essential_tech1)

    essential_tech1 = essential_tech_box[2].find('pre').text
    major_work_list_tmp.append(essential_tech1)


    essential_tech2 = essential_tech_box[3].find('pre').text
    major_work_list_tmp.append(essential_tech2)

    tech_list.append(major_work_list_tmp)



    
    

In [9]:
print(len(company_name_list))
print(len(hire_list))
print(len(tech_list))
print(len(major_work_list))
print(len(company_address_list))

148
148
148
148
148


In [10]:
import pandas as pd

# 새로운 DataFrame 생성
new_data = pd.DataFrame({
    
    '기업이름' : company_name_list,
    '고용형태': hire_list,
    '연봉' : salary_list,
    '기업주소': company_address_list,
    '필요역량':tech_list
})

# 기존의 CSV 파일 불러오기
job_data = pd.read_csv('../data/jumpit_deeplearning_base.csv')

# 두 DataFrame 병합
merged_data = pd.concat([job_data, new_data], axis=1)


# 결과 저장
merged_data.to_csv('../data/jumpit_deeplearning_base2.csv', index=False)


In [11]:
merged_data.head()


,Unnamed: 0,키워드,url,기업이름,고용형태,연봉,기업주소,담당업무,필요역량
0,0,딥러닝,/position/19405,클레온,경력경력 2~10년,회사내규에 따름,근무지역서울 중구 소월로2길 30 티타워 18층 클레온지도보기·주소복사,• 딥러닝 네트워크 고도화\n - CV : Computer Vision\n - Ge...,"[DeepLearningTensorFlowPyTorch, • 딥러닝 혹은 유사 분야..."
1,1,딥러닝,/position/19117,에프에스솔루션,경력경력 10~15년,회사내규에 따름,근무지역경기 성남시 수정구 금토로80번길 11지도보기·주소복사,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
2,2,딥러닝,/position/19114,에프에스솔루션,경력경력 3~9년,회사내규에 따름,근무지역경기 성남시 수정구 금토로80번길 11지도보기·주소복사,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
3,3,딥러닝,/position/19113,에프에스솔루션,경력신입,회사내규에 따름,근무지역경기 성남시 수정구 금토로80번길 11지도보기·주소복사,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
4,4,딥러닝,/position/22057,슈어소프트테크,경력경력 3~15년,회사내규에 따름,근무지역경기 성남시 수정구 금토로80번길 37 인피니티타워 E동지도보기·주소복사,• 머신러닝/딥러닝 관련 개발\n• 머신러닝/딥러닝 서비스 운영 시스템 설계 및 구현,[AI/인공지능C++MachineLearningDeepLearningNLPTenso...


In [12]:
# 컬럼 재정렬 및 불필요한 컬럼 제거
merged_data = merged_data[['기업이름', '고용형태', '연봉', '기업주소', '키워드', '담당업무','필요역량']]
merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,클레온,경력경력 2~10년,회사내규에 따름,근무지역서울 중구 소월로2길 30 티타워 18층 클레온지도보기·주소복사,딥러닝,• 딥러닝 네트워크 고도화\n - CV : Computer Vision\n - Ge...,"[DeepLearningTensorFlowPyTorch, • 딥러닝 혹은 유사 분야..."
1,에프에스솔루션,경력경력 10~15년,회사내규에 따름,근무지역경기 성남시 수정구 금토로80번길 11지도보기·주소복사,딥러닝,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
2,에프에스솔루션,경력경력 3~9년,회사내규에 따름,근무지역경기 성남시 수정구 금토로80번길 11지도보기·주소복사,딥러닝,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
3,에프에스솔루션,경력신입,회사내규에 따름,근무지역경기 성남시 수정구 금토로80번길 11지도보기·주소복사,딥러닝,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
4,슈어소프트테크,경력경력 3~15년,회사내규에 따름,근무지역경기 성남시 수정구 금토로80번길 37 인피니티타워 E동지도보기·주소복사,딥러닝,• 머신러닝/딥러닝 관련 개발\n• 머신러닝/딥러닝 서비스 운영 시스템 설계 및 구현,[AI/인공지능C++MachineLearningDeepLearningNLPTenso...


In [13]:
# 결과 저장
merged_data.to_csv('../data/project_eda_dataset/jumpit_deeplearning.csv', index=False)

In [14]:
merged_data['기업주소'] = merged_data['기업주소'].apply(lambda x: x.split('지도보기·주소복사')[0].replace('근무지역', '').strip())

# 결과 확인 및 저장
merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,클레온,경력경력 2~10년,회사내규에 따름,서울 중구 소월로2길 30 티타워 18층 클레온,딥러닝,• 딥러닝 네트워크 고도화\n - CV : Computer Vision\n - Ge...,"[DeepLearningTensorFlowPyTorch, • 딥러닝 혹은 유사 분야..."
1,에프에스솔루션,경력경력 10~15년,회사내규에 따름,경기 성남시 수정구 금토로80번길 11,딥러닝,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
2,에프에스솔루션,경력경력 3~9년,회사내규에 따름,경기 성남시 수정구 금토로80번길 11,딥러닝,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
3,에프에스솔루션,경력신입,회사내규에 따름,경기 성남시 수정구 금토로80번길 11,딥러닝,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
4,슈어소프트테크,경력경력 3~15년,회사내규에 따름,경기 성남시 수정구 금토로80번길 37 인피니티타워 E동,딥러닝,• 머신러닝/딥러닝 관련 개발\n• 머신러닝/딥러닝 서비스 운영 시스템 설계 및 구현,[AI/인공지능C++MachineLearningDeepLearningNLPTenso...


In [15]:
merged_data['고용형태'] = merged_data['고용형태'].str.replace('경력경력', '경력')

In [16]:
merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,클레온,경력 2~10년,회사내규에 따름,서울 중구 소월로2길 30 티타워 18층 클레온,딥러닝,• 딥러닝 네트워크 고도화\n - CV : Computer Vision\n - Ge...,"[DeepLearningTensorFlowPyTorch, • 딥러닝 혹은 유사 분야..."
1,에프에스솔루션,경력 10~15년,회사내규에 따름,경기 성남시 수정구 금토로80번길 11,딥러닝,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
2,에프에스솔루션,경력 3~9년,회사내규에 따름,경기 성남시 수정구 금토로80번길 11,딥러닝,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
3,에프에스솔루션,경력신입,회사내규에 따름,경기 성남시 수정구 금토로80번길 11,딥러닝,• 딥러닝/머신러닝 모델 개발\n - Classification(분류) / Obj...,[OpenCVC++MachineLearningDeepLearningTensorFlo...
4,슈어소프트테크,경력 3~15년,회사내규에 따름,경기 성남시 수정구 금토로80번길 37 인피니티타워 E동,딥러닝,• 머신러닝/딥러닝 관련 개발\n• 머신러닝/딥러닝 서비스 운영 시스템 설계 및 구현,[AI/인공지능C++MachineLearningDeepLearningNLPTenso...


In [40]:
merged_data.to_csv('../data/project_eda_dataset/jumpit_deeplearning.csv', index=False)